# Toy Induction (attn-only-2l): Drift via Circuit Competition

Reproduce martingale drift (order sensitivity) in a minimal circuit where induction heads can exist, and show that ablating those heads collapses the drift toward an order-invariant counting baseline.

In [1]:
# Notebook path setup: make repo imports work regardless of where you run this from
from pathlib import Path
import sys

cwd = Path.cwd().resolve()
repo_candidates = [cwd, cwd.parent]
repo_root = next((p for p in repo_candidates if (p / 'bayesian_llm').exists()), None)
if repo_root is None:
    raise RuntimeError(f'Could not find repo root from cwd={cwd}.')

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print('Repo root:', repo_root)


Repo root: /root/bayesian-llm


In [3]:
# Load attn-only-2l (TransformerLens)
import torch
import numpy as np
import pandas as pd

from transformer_lens import HookedTransformer

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DTYPE = torch.float32 if DEVICE == 'cpu' else torch.float16

model = HookedTransformer.from_pretrained('attn-only-2l', device=DEVICE, dtype=DTYPE)
print('Loaded attn-only-2l')
print('device:', DEVICE, 'dtype:', DTYPE)
print('n_layers:', model.cfg.n_layers, 'n_heads:', model.cfg.n_heads, 'd_model:', model.cfg.d_model)


Loaded pretrained model attn-only-2l into HookedTransformer
Loaded attn-only-2l
device: cuda dtype: torch.float16
n_layers: 2 n_heads: 8 d_model: 512


In [4]:
# Define the stimuli and verify tokenization (important!)

# We predict between exactly two single-token candidates: " X" and " Y"
X_STR = ' X'
Y_STR = ' Y'

X_ID = model.to_single_token(X_STR)
Y_ID = model.to_single_token(Y_STR)

print('X_STR token id:', X_ID)
print('Y_STR token id:', Y_ID)

# Sanity: decode
print('decode X:', repr(model.tokenizer.decode([X_ID])))
print('decode Y:', repr(model.tokenizer.decode([Y_ID])))


X_STR token id: 1558
Y_STR token id: 712
decode X: ' X'
decode Y: ' Y'


In [5]:
# Construct sequences with identical counts but different order

# All have 3 X and 3 Y. If the model were a perfect count-based Bayesian updater, predictions should match.
SEQUENCES = {
    'patterned': ['X','Y','X','Y','X','Y'],
    'clumped':   ['X','X','X','Y','Y','Y'],
    'mixed':     ['X','Y','Y','X','X','Y'],
}

def counts(seq):
    n_x = sum(t == 'X' for t in seq)
    n_y = sum(t == 'Y' for t in seq)
    return n_x, n_y

for name, seq in SEQUENCES.items():
    print(name, seq, 'counts=', counts(seq))


patterned ['X', 'Y', 'X', 'Y', 'X', 'Y'] counts= (3, 3)
clumped ['X', 'X', 'X', 'Y', 'Y', 'Y'] counts= (3, 3)
mixed ['X', 'Y', 'Y', 'X', 'X', 'Y'] counts= (3, 3)


In [6]:
# Order-invariant baselines: Frequentist and Bayesian (Beta-Bernoulli)

# Frequentist: p_hat = n_X / (n_X + n_Y)
# Bayesian posterior predictive: Beta(alpha,beta) prior -> (alpha+n_X)/(alpha+beta+n)

ALPHA = 1.0
BETA = 1.0

def frequentist_p_x(n_x, n_y):
    n = n_x + n_y
    return n_x / n if n > 0 else 0.5

def bayes_p_x(n_x, n_y, *, alpha=ALPHA, beta=BETA):
    n = n_x + n_y
    return (alpha + n_x) / (alpha + beta + n)

# These should be identical across sequences (same counts).
for name, seq in SEQUENCES.items():
    n_x, n_y = counts(seq)
    print(name, 'freq=', round(frequentist_p_x(n_x,n_y),4), 'bayes=', round(bayes_p_x(n_x,n_y),4))


patterned freq= 0.5 bayes= 0.5
clumped freq= 0.5 bayes= 0.5
mixed freq= 0.5 bayes= 0.5


In [7]:
# Model prediction utilities: logits/probs for next token

import torch

def make_prompt(seq_tokens):
    # Force each token to be preceded by a space by using "Sequence: " prefix.
    return 'Sequence: ' + ' '.join(seq_tokens)

@torch.no_grad()
def model_p_x(prompt: str):
    # Returns normalized P(X | {X,Y}), plus raw logits and softmax probs for the two tokens.
    logits = model(prompt)  # [batch=1, pos, vocab]
    last = logits[0, -1, :]

    logit_x = float(last[X_ID].item())
    logit_y = float(last[Y_ID].item())

    probs = torch.softmax(last.float(), dim=-1)
    p_x = float(probs[X_ID].item())
    p_y = float(probs[Y_ID].item())

    norm = p_x + p_y
    p_x_norm = p_x / norm if norm > 0 else 0.5
    p_y_norm = p_y / norm if norm > 0 else 0.5

    return {
        'logit_X': logit_x,
        'logit_Y': logit_y,
        'logit_diff_XminusY': logit_x - logit_y,
        'p_X_raw': p_x,
        'p_Y_raw': p_y,
        'p_X_norm': p_x_norm,
        'p_Y_norm': p_y_norm,
    }

# Quick smoke test
print(model_p_x(make_prompt(SEQUENCES['patterned'])))


{'logit_X': 16.453125, 'logit_Y': 16.671875, 'logit_diff_XminusY': -0.21875, 'p_X_raw': 0.3649851381778717, 'p_Y_raw': 0.454231321811676, 'p_X_norm': 0.44552954683372514, 'p_Y_norm': 0.5544704531662749}


In [8]:
# Baseline: evaluate patterned/clumped/mixed and compute drift

rows = []
for name, seq in SEQUENCES.items():
    n_x, n_y = counts(seq)
    prompt = make_prompt(seq)
    out = model_p_x(prompt)

    rows.append({
        'name': name,
        'seq': ' '.join(seq),
        'n_X': n_x,
        'n_Y': n_y,
        'freq_p_X': frequentist_p_x(n_x,n_y),
        'bayes_p_X': bayes_p_x(n_x,n_y),
        **out,
    })

df = pd.DataFrame(rows).sort_values('name')
display(df)

drift = float(df.p_X_norm.max() - df.p_X_norm.min())
print('Drift (max-min) over permutations with same counts:', drift)


,name,seq,n_X,n_Y,freq_p_X,bayes_p_X,logit_X,logit_Y,logit_diff_XminusY,p_X_raw,p_Y_raw,p_X_norm,p_Y_norm
1,clumped,X X X Y Y Y,3,3,0.5,0.5,13.710938,16.484375,-2.773438,0.046506,0.744730,0.058777,0.941223
2,mixed,X Y Y X X Y,3,3,0.5,0.5,16.281250,17.015625,-0.734375,0.271795,0.566470,0.324235,0.675765
0,patterned,X Y X Y X Y,3,3,0.5,0.5,16.453125,16.671875,-0.218750,0.364985,0.454231,0.445530,0.554470


Drift (max-min) over permutations with same counts: 0.38675298854420087


## Visualize Attention with CircuitVis

We visualize attention patterns for both layers. Induction heads typically show a distinctive pattern where they attend to positions that enable copying of the next token after a previous occurrence of the same context token.


In [11]:
# Run with cache and visualize attention patterns for each sequence

import pkgutil
from IPython.display import display

if pkgutil.find_loader('circuitsvis') is None:
    raise RuntimeError('circuitsvis not installed. Run: pip install circuitsvis')

import circuitsvis as cv

for name, seq in SEQUENCES.items():
    prompt = make_prompt(seq)
    logits, cache = model.run_with_cache(prompt)
    tokens = model.to_str_tokens(prompt)

    print('===', name, '===')
    print('Tokens:', tokens)

    for layer in range(model.cfg.n_layers):
        # cache['pattern', layer] has shape [batch, head, query, key]
        patt = cache['pattern', layer][0].detach().cpu().numpy()
        display(cv.attention.attention_patterns(tokens=tokens, attention=patt))


=== patterned ===
Tokens: ['<|BOS|>', 'Sequence', ':', ' X', ' Y', ' X', ' Y', ' X', ' Y']


=== clumped ===
Tokens: ['<|BOS|>', 'Sequence', ':', ' X', ' X', ' X', ' Y', ' Y', ' Y']


=== mixed ===
Tokens: ['<|BOS|>', 'Sequence', ':', ' X', ' Y', ' Y', ' X', ' X', ' Y']


## Identify Induction Heads via Ablation Sweep

We use a purely behavioral criterion: *which head ablation reduces drift the most?*

Metric: drift = max(P(X|{X,Y})) - min(P(X|{X,Y})) across the three sequences.


In [12]:
# Ablation utilities

from transformer_lens.utils import get_act_name

def ablate_heads(heads):
    # heads: list of (layer, head)
    # Hook on z so the head contributes ~0 to residual stream.
    layer_to_heads = {}
    for layer, head in heads:
        layer_to_heads.setdefault(int(layer), []).append(int(head))

    hooks = []
    for layer, hs in layer_to_heads.items():
        hs = sorted(set(hs))
        hook_name = get_act_name('z', layer)

        def hook_fn(z, hook, hs=hs):
            z[:, :, hs, :] = 0.0
            return z

        hooks.append((hook_name, hook_fn))
    return hooks

@torch.no_grad()
def eval_sequences(fwd_hooks=None):
    rows = []
    for name, seq in SEQUENCES.items():
        n_x, n_y = counts(seq)
        prompt = make_prompt(seq)
        logits = model(prompt) if fwd_hooks is None else model.run_with_hooks(prompt, fwd_hooks=fwd_hooks)
        last = logits[0, -1, :]
        probs = torch.softmax(last.float(), dim=-1)
        p_x = float(probs[X_ID].item())
        p_y = float(probs[Y_ID].item())
        p_x_norm = p_x / (p_x + p_y) if (p_x + p_y) > 0 else 0.5

        rows.append({
            'name': name,
            'p_X_norm': p_x_norm,
            'logit_diff_XminusY': float(last[X_ID].item() - last[Y_ID].item()),
            'bayes_p_X': bayes_p_x(n_x,n_y),
            'freq_p_X': frequentist_p_x(n_x,n_y),
        })
    df = pd.DataFrame(rows)
    drift = float(df.p_X_norm.max() - df.p_X_norm.min())
    mae_to_bayes = float((df.p_X_norm - df.bayes_p_X).abs().mean())
    return df.sort_values('name'), drift, mae_to_bayes

base_df, base_drift, base_mae = eval_sequences()
print('Baseline drift:', base_drift)
print('Baseline MAE to Bayes:', base_mae)
display(base_df)


Baseline drift: 0.38675298854420087
Baseline MAE to Bayes: 0.22381949730340045


,name,p_X_norm,logit_diff_XminusY,bayes_p_X,freq_p_X
1,clumped,0.058777,-2.773438,0.5,0.5
2,mixed,0.324235,-0.734375,0.5,0.5
0,patterned,0.445530,-0.218750,0.5,0.5


In [13]:
# Single-head ablation sweep: which head kills drift?

results = []
for layer in range(model.cfg.n_layers):
    for head in range(model.cfg.n_heads):
        hooks = ablate_heads([(layer, head)])
        _, drift, mae = eval_sequences(fwd_hooks=hooks)
        results.append({
            'layer': layer,
            'head': head,
            'drift': drift,
            'mae_to_bayes': mae,
            'drift_reduction': base_drift - drift,
        })

df_sweep = pd.DataFrame(results).sort_values('drift_reduction', ascending=False)
display(df_sweep.head(10))

best = df_sweep.iloc[0].to_dict()
print('Top drift-reducing head:', best)


,layer,head,drift,mae_to_bayes,drift_reduction
14,1,6,0.032705,0.101044,0.354048
3,0,3,0.035071,0.021453,0.351682
5,0,5,0.160650,0.367167,0.226103
0,0,0,0.194464,0.117041,0.192289
15,1,7,0.200076,0.382442,0.186677
6,0,6,0.287796,0.211990,0.098957
9,1,1,0.348380,0.265338,0.038373
12,1,4,0.383625,0.229052,0.003128
13,1,5,0.408887,0.196189,-0.022134
10,1,2,0.422049,0.196773,-0.035296


Top drift-reducing head: {'layer': 1.0, 'head': 6.0, 'drift': 0.03270505311883354, 'mae_to_bayes': 0.10104366859481391, 'drift_reduction': 0.35404793542536733}


In [14]:
# Choose induction-head candidates and re-test

TOPK = 2  # increase if drift remains
candidates = [(int(r.layer), int(r.head)) for r in df_sweep.head(TOPK).itertuples()]
print('Ablating candidates:', candidates)

hooks = ablate_heads(candidates)
abl_df, abl_drift, abl_mae = eval_sequences(fwd_hooks=hooks)
print('After ablation drift:', abl_drift, '(baseline:', base_drift, ')')
print('After ablation MAE to Bayes:', abl_mae, '(baseline:', base_mae, ')')
display(abl_df)


Ablating candidates: [(1, 6), (0, 3)]
After ablation drift: 0.005970507201289887 (baseline: 0.38675298854420087 )
After ablation MAE to Bayes: 0.17803790194277358 (baseline: 0.22381949730340045 )


,name,p_X_norm,logit_diff_XminusY,bayes_p_X,freq_p_X
1,clumped,0.674908,0.730469,0.5,0.5
2,mixed,0.678327,0.746094,0.5,0.5
0,patterned,0.680879,0.757812,0.5,0.5


In [ ]:
# Visualize attention again (attention *patterns* won't change under z-ablation)

import pkgutil
from IPython.display import display

if pkgutil.find_loader('circuitsvis') is None:
    raise RuntimeError('circuitsvis not installed. Run: pip install circuitsvis')

import circuitsvis as cv

name = 'patterned'
seq = SEQUENCES[name]
prompt = make_prompt(seq)

logits, cache = model.run_with_cache(prompt)
tokens = model.to_str_tokens(prompt)

print('Candidates:', candidates)
print('Note: z-ablation changes contributions, not the attention weights themselves.')

for layer in range(model.cfg.n_layers):
    patt = cache['pattern', layer][0].detach().cpu().numpy()
    display(cv.attention.attention_patterns(tokens=tokens, attention=patt))


Candidates: [(1, 6), (0, 3)]
Note: z-ablation changes contributions, not the attention weights themselves.


We now evaluate the ablated attn-only-2l on tasks similar to `toy_example.ipynb`:
1) evidence sweep vs Bayes for the A=50% vs B=75% generator task,
2) martingale/order drift for fixed counts,
3) optional real-time tracking of a hidden bias.

In [ ]:
# Prompt helpers + prediction wrapper

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

from bayesian_llm.bayes import two_generator_posterior_predictive

PROMPT_STYLE = 'toy'  # 'toy' | 'minimal'
N_TOTAL = 10
N_PERMUTATIONS = 10


def toy_prompt(sequence_tokens, *, control: str = 'order_irrelevant'):
    seq_str = ' '.join(sequence_tokens)
    if control == 'base':
        prefix = 'Two random generators. Generator A: 50% X. Generator B: 75% X.'
    elif control == 'independent':
        prefix = 'Two random generators. Generator A: 50% X. Generator B: 75% X. Draws are independent.'
    elif control == 'order_irrelevant':
        prefix = 'Two random generators. Generator A: 50% X. Generator B: 75% X. Draws are independent. Order does not matter.'
    else:
        raise ValueError(control)

    return f"{prefix} Sequence: {seq_str}. Predict the next output (X or Y):"


def minimal_prompt(sequence_tokens):
    return 'Sequence: ' + ' '.join(sequence_tokens) + ' Next is:'


def make_prompt(seq):
    if PROMPT_STYLE == 'toy':
        return toy_prompt(seq, control='order_irrelevant')
    if PROMPT_STYLE == 'minimal':
        return minimal_prompt(seq)
    raise ValueError('Unknown PROMPT_STYLE')


@torch.no_grad()
def p_x_for_prompt(prompt, *, fwd_hooks=None):
    logits = model(prompt) if fwd_hooks is None else model.run_with_hooks(prompt, fwd_hooks=fwd_hooks)
    last = logits[0, -1, :]
    probs = torch.softmax(last.float(), dim=-1)
    p_x = float(probs[X_ID].item())
    p_y = float(probs[Y_ID].item())
    return p_x / (p_x + p_y) if (p_x + p_y) > 0 else 0.5


# Hooks from the ablation sweep earlier in this notebook
try:
    _ = hooks
except NameError:
    hooks = None
    print('hooks not defined: run the ablation sweep above to define `hooks` and `candidates`.')

print('PROMPT_STYLE:', PROMPT_STYLE)
print('Using ablation hooks:', hooks is not None)


In [ ]:
# Evidence sweep: baseline vs ablated

from bayesian_llm.data import make_sequence, permute_sequence

rows = []

for n_x in range(N_TOTAL + 1):
    base_seq = make_sequence(n_x=n_x, n_total=N_TOTAL, x='X', y='Y')
    true_p = two_generator_posterior_predictive(n_x=n_x, n_total=N_TOTAL)
    freq_p = n_x / N_TOTAL if N_TOTAL > 0 else 0.5

    preds_base = []
    preds_abl = []
    for _ in range(N_PERMUTATIONS):
        seq = permute_sequence(base_seq, rng=np.random.default_rng(0))
        prompt = make_prompt(seq)
        preds_base.append(p_x_for_prompt(prompt, fwd_hooks=None))
        if hooks is not None:
            preds_abl.append(p_x_for_prompt(prompt, fwd_hooks=hooks))

    rows.append({
        'n_X': n_x,
        'bayes_p_X': true_p,
        'freq_p_X': freq_p,
        'baseline_mean': float(np.mean(preds_base)),
        'baseline_std': float(np.std(preds_base, ddof=1) if len(preds_base)>1 else 0.0),
        'ablated_mean': float(np.mean(preds_abl)) if preds_abl else None,
        'ablated_std': float(np.std(preds_abl, ddof=1) if len(preds_abl)>1 else None),
    })

df_sweep = pd.DataFrame(rows)
display(df_sweep)

# Plot
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(df_sweep.n_X, df_sweep.bayes_p_X, color='black', linewidth=2, label='Bayes (A=0.5 vs B=0.75)')
ax.plot(df_sweep.n_X, df_sweep.freq_p_X, color='gray', linestyle='--', label='Frequentist count')
ax.errorbar(df_sweep.n_X, df_sweep.baseline_mean, yerr=df_sweep.baseline_std, marker='o', capsize=3, label='attn-only-2l (baseline)')
if df_sweep.ablated_mean.notna().all():
    ax.errorbar(df_sweep.n_X, df_sweep.ablated_mean, yerr=df_sweep.ablated_std, marker='x', capsize=3, label='attn-only-2l (ablated)')
ax.set_title(f'Evidence sweep ({PROMPT_STYLE} prompt)')
ax.set_xlabel('n_X')
ax.set_ylabel('P(next is X | {X,Y})')
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)
ax.legend()
plt.show()

# Summary stats
mae_base = float((df_sweep.baseline_mean - df_sweep.bayes_p_X).abs().mean())
print('Baseline MAE to Bayes:', mae_base)
if df_sweep.ablated_mean.notna().all():
    mae_abl = float((df_sweep.ablated_mean - df_sweep.bayes_p_X).abs().mean())
    print('Ablated MAE to Bayes:', mae_abl)


In [ ]:
# Martingale/order drift for fixed counts

from bayesian_llm.data import permute_sequence

seq_fixed = ['X','X','X','X','Y']  # 4X/1Y
N_SAMPLES = 50

base_preds = []
abl_preds = []
for _ in range(N_SAMPLES):
    seq = permute_sequence(seq_fixed, rng=np.random.default_rng())
    prompt = make_prompt(seq)
    base_preds.append(p_x_for_prompt(prompt))
    if hooks is not None:
        abl_preds.append(p_x_for_prompt(prompt, fwd_hooks=hooks))

base_drift = float(max(base_preds) - min(base_preds))
print('Baseline drift:', base_drift)

if abl_preds:
    abl_drift = float(max(abl_preds) - min(abl_preds))
    print('Ablated drift:', abl_drift)

print('Baseline preds (first 10):', [round(x,4) for x in base_preds[:10]])
if abl_preds:
    print('Ablated preds (first 10):', [round(x,4) for x in abl_preds[:10]])


In [ ]:
# Optional: real-time tracking of a hidden bias (toy_example-style)

import random

MAX_SEQ_LEN = 20
HIDDEN_BIAS = 0.75  # true generator favors X

sequence = []
rows = []

for t in range(MAX_SEQ_LEN):
    # Generate the world
    token = 'X' if random.random() < HIDDEN_BIAS else 'Y'
    sequence.append(token)

    # Bayesian predictive using two-generator task (A=0.5 vs B=0.75)
    n_x = sum(1 for x in sequence if x == 'X')
    n_total = len(sequence)
    bayes_p = two_generator_posterior_predictive(n_x=n_x, n_total=n_total)

    # Frequentist count (order-invariant)
    freq_p = n_x / n_total if n_total > 0 else 0.5

    # Model predictions
    prompt = make_prompt(sequence)
    p_base = p_x_for_prompt(prompt)
    p_abl = p_x_for_prompt(prompt, fwd_hooks=hooks) if hooks is not None else None

    rows.append({
        't': t + 1,
        'token': token,
        'bayes_p_X': bayes_p,
        'freq_p_X': freq_p,
        'baseline_p_X': p_base,
        'ablated_p_X': p_abl,
    })

track = pd.DataFrame(rows)
display(track.tail(5))

# Display generated sequence in the output
print('Generated sequence:', ' '.join(sequence))

fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(track.t, track.bayes_p_X, marker='o', label='Bayes')
ax.plot(track.t, track.freq_p_X, marker='s', linestyle='--', label='Frequentist count')
ax.plot(track.t, track.baseline_p_X, marker='x', label='attn-only-2l (baseline)')
if track.ablated_p_X.notna().all():
    ax.plot(track.t, track.ablated_p_X, marker='^', label='attn-only-2l (ablated)')
ax.axhline(HIDDEN_BIAS, color='gray', linestyle='--', alpha=0.6, label='True bias')
ax.set_title(f'Real-time tracking (prompt={PROMPT_STYLE})')
ax.set_xlabel('time step')
ax.set_ylabel('P(next is X | {X,Y})')
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)
ax.legend()
plt.show()
